In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
import string
from nltk.stem.snowball import SnowballStemmer

import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
# Reading the data file into a DATAFRAME and checking the shape 
data=pd.read_csv('35.1AmazonMobileDataUncleaned.csv')
print(data.shape)

(71922, 2)


# Data Cleaning(Text)

In [3]:
def datapreprocess(sen):
    
    sen = re.sub(r"didn't", "did not", sen)
    sen = re.sub(r"don't", "do not", sen)
    sen = re.sub(r"won't", "will not", sen)
    sen = re.sub(r"can't", "can not", sen)
    sen = re.sub(r"wasn't", "was not", sen)
    sen = re.sub(r"\'ve", " have", sen)
    sen = re.sub(r"\'m", " am", sen)
    sen = re.sub(r"\'ll", " will", sen)
    sen = re.sub(r"\'re", " are", sen)
    sen = re.sub(r"\'s", " is", sen)
    sen = re.sub(r"\'d", " would", sen)
    sen = re.sub(r"\'t", " not", sen)
    sen = re.sub(r"\'m", " am", sen)
    
    
    p = set(string.punctuation) # p take all the punctuations, punctuations displayed below
    sen=sen.lower() # convert text to lower case
    words=sen.split() # split the text into words
    ctext=[]
    for i in range(10): #adding numbers from 0 - 9 to p 
        p.add(str(i))
    for i in words:
        t=''.join([x for x in i.encode("ascii","ignore").decode("ascii") if x not in p]) # ignoring non ascii charecters and numbers
        ctext.append(t)
    return " ".join([i for i in ctext]) # joining the cleaned words to text
    
    

stop=set(stopwords.words('english'))
print(stop)
stop.remove('no')
stop.remove('not')
stop.remove('nor')

preprocessed_essays = []
for sentance in data['uncleanedreview'].values:
    try:
        ctxt=datapreprocess(str(sentance))
        if len(ctxt)==0: raise()
        ctxt = ' '.join(e for e in ctxt.split() if e not in stop)
        preprocessed_essays.append(ctxt.lower().strip())
    except:
        preprocessed_essays.append("NAN")  
data['cleanedtext'] = preprocessed_essays
data.drop(data[data["cleanedtext"]=="NAN"].index,axis = 0,inplace = True) # deleting rows that have no text 
data=data.reset_index(drop='true') # after deleting rows index should be reset
data['decision']= data['decision'].replace('positive',1)
data['decision']= data['decision'].replace('negative',0)


s = SnowballStemmer("english")
p=[]
for i in data['cleanedtext'].values:
    k=[]
    for word in i.split():
        k.append(s.stem(word))
    l=' '.join(k)
    p.append(l)

data['finaltext']=p

data.to_csv (r'AmazonMobileDataCleaned.csv')

{'in', 'i', 'm', "haven't", 'shouldn', 'ma', "hasn't", "isn't", 'this', "won't", 'had', 'such', 'my', "that'll", "didn't", 'won', 'an', 'here', 'don', 'then', 's', 'ourselves', "should've", 'isn', 'no', 'few', 'each', 'shan', 'or', 'he', 'before', 'some', 'should', 'our', 'couldn', "needn't", 'its', 'his', 'both', 'off', 'for', 't', 'haven', "you'd", 'only', "hadn't", 'nor', 'too', 'her', 'were', "couldn't", 'after', 'between', 'most', 'about', 'against', 'can', 'other', "you're", 'until', 're', 'be', 'them', 'mustn', "she's", 'because', 'doesn', 'while', 'during', 'these', 'having', 'why', 'wasn', 'itself', 'herself', 'did', 'of', "don't", 'was', 'weren', "weren't", 'how', 'than', "aren't", 'what', 'needn', 've', 'ain', 'over', 'hers', "shan't", 'am', "wouldn't", 'does', 'him', 'me', 'ours', 'under', 'didn', 'those', 'once', "doesn't", 'will', 'aren', 'doing', 'wouldn', 'yourself', 'now', 'y', 'a', 'has', 'above', 'below', 'your', 'out', 'yourselves', 'they', 'that', 'it', 'the', "mig

# Data splitting into train,validaion&test datasets

In [4]:
data=pd.read_csv('AmazonMobileDataCleaned.csv')

label = data['decision'].values #output
feedback = data.drop(['decision'], axis=1) #inputs


from sklearn.model_selection import train_test_split
inputtrain, inputtest, outputtrain, outputtest = train_test_split(feedback, label, test_size=0.20, stratify=label)
inputtrain, inputcv, outputtrain, outputcv = train_test_split(inputtrain, outputtrain, test_size=0.20, stratify=outputtrain)
print(inputtrain.shape)
print(inputcv.shape)
print(inputtest.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)

(46029, 4)
(11508, 4)
(14385, 4)
(46029,)
(11508,)
(14385,)


# Data Preprocessing on Training,CV&Testing datasets

In [5]:
#fit() applied only on inputtrain datasets, transform() applied on inputtrain,inputtest,inputcv datasets
#fit() & transform() applied on each & every column(features). Here we have only one column 'finaltext'

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(min_df=10,ngram_range=(1,4))
vec.fit(inputtrain['finaltext'].values) # fit has to happen only on train data


train_cleanedtext = vec.transform(inputtrain['finaltext'].values)
cv_cleanedtext = vec.transform(inputcv['finaltext'].values)
test_cleanedtext= vec.transform(inputtest['finaltext'].values)

print("After vectorizations")
print(train_cleanedtext.shape)
print(cv_cleanedtext.shape)
print(test_cleanedtext.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)
print("...............................")
print(len(vec.get_feature_names()))

After vectorizations
(46029, 23958)
(11508, 23958)
(14385, 23958)
(46029,)
(11508,)
(14385,)
...............................
['abandon', 'abe', 'abil', 'abil add', 'abil use', 'abl', 'abl access', 'abl activ', 'abl activ phone', 'abl add', 'abl buy', 'abl call', 'abl chang', 'abl charg', 'abl connect', 'abl disabl', 'abl download', 'abl easili', 'abl figur', 'abl find', 'abl fix', 'abl get', 'abl go', 'abl handl', 'abl hear', 'abl help', 'abl keep', 'abl make', 'abl make call', 'abl move', 'abl put', 'abl receiv', 'abl return', 'abl run', 'abl see', 'abl send', 'abl set', 'abl take', 'abl tell', 'abl transfer', 'abl turn', 'abl unlock', 'abl upgrad', 'abl use', 'abl use phone', 'abl work', 'abroad', 'absenc', 'absolut', 'absolut amaz', 'absolut beauti', 'absolut best', 'absolut fantast', 'absolut great', 'absolut hate', 'absolut horribl', 'absolut incred', 'absolut love', 'absolut love phone', 'absolut no', 'absolut no complaint', 'absolut no issu', 'absolut no problem', 'absolut not',

In [8]:
print(len(vec.get_feature_names()))

23958
